# REWOO-ReAct Hybrid Pattern Airline Assistant

## Overview
In this example we will guide you through how to create a REWOO-ReAct hybrid pattern implementation using Strands multiagent orchestration. We will demonstrate a two-agent system that combines REWOO's structured planning approach with ReAct's direct tool execution, creating an efficient hybrid workflow that balances planning reliability with execution flexibility for airline customer service tasks.

## Agent Details
<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|Native tools used   |generate_flight_plan, solve_flight_query          |
|Custom tools created|Two core hybrid tools with airline domain integration|
|Agent Structure     |Two-agent sequential pipeline (Planner → Solver)  |
|AWS services used   |Amazon Bedrock (Claude 3.7 Sonnet)                |
|Domain tools        |MAbench airline tools (booking, search, updates)  |

</div>

## Architecture
<div style="text-align:center">
    <img src="../images/rewoo-react.png" alt="REWOO-ReAct Hybrid Architecture" width="600">
    <p>The system consists of two specialized agents connected in a sequential graph:</p>
    <p><em>Hybrid Architecture: User Query → [Planner] → [Solver] → Final Response</em></p>
</div>

## Key Features
* **Two-agent hybrid pipeline**: Planner generates structured plans, Solver executes with ReAct flexibility
* **Best of both worlds**: REWOO's planning reliability combined with ReAct's execution adaptability
* **Reduced latency**: Only 2 agents vs 3 in pure REWOO implementation
* **Structured execution**: Plans use #E1, #E2, #E3 format for guided tool execution
* **ReAct flexibility**: Solver can adapt execution based on intermediate results and policy constraints
* **Better error handling**: ReAct solver can recover from plan deviations and policy violations
* **Plan-guided execution**: Maintains systematic approach while allowing execution flexibility
* **Airline domain integration**: Complete integration with MAbench airline tools and TauBench datasets




In [ ]:
!pip3 install -r ./requirements.txt --quiet --upgrade
!pip3 install strands-agents strands-agents-tools --quiet
!pip3 install litellm


## Import Dependencies

Now let's import all the necessary libraries and modules for our hybrid implementation. We import the Strands framework components so that we can build custom agents and multiagent graphs, plus all the standard libraries needed for data processing and AWS Bedrock integration.

In [ ]:
import time, os, json, ast, uuid, logging
from strands import Agent, tool
from strands.models import BedrockModel
from strands.multiagent.graph import GraphBuilder
from strands.agent import AgentResult
from strands.types.content import Message
from strands.telemetry.metrics import EventLoopMetrics
from helpers.rewoo_react_helper_funcs import *
from helpers.bedrock_helper import get_bedrock_response, get_claude_response, get_claude_response_text


## Configure Strands Framework

Now let's set up the core Strands framework components that will power our multiagent system built on hybrid orchestration. We need to configure the AWS Bedrock connection, conversation management, and logging to ensure our three-agent pipeline runs smoothly.

### Framework Setup Process

First, we'll establish the **AWS region** and create a `BedrockModel` instance that all agents (in the hybrid system  will share. We do this so that all agents use the same LLM configuration for consistent behavior. 

Finally, we'll configure **logging** to minimize noise during execution so we can focus on the rewoo-react execution flow and results.


In [ ]:
#Clients

region = "us-east-1"

logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)



# Create BedrockModel with specified region
bedrock_model_taubench = BedrockModel(region_name= region)

#setup logging
# Disable all logging except critical errors
logging.basicConfig(level=logging.CRITICAL)

# Silence specific noisy loggers completely
for logger_name in ["strands", "graph", "event_loop", "registry", "sliding_window_conversation_manager", "bedrock", "streaming"]:
    logging.getLogger(logger_name).setLevel(logging.CRITICAL)


## Import airline domain tools

Now we'll import the comprehensive set of airline domain tools from MAbench and TauBench. These tools provide the actual functionality that our REWOO Worker agent will execute, including flight booking, reservation management, and customer service operations.

In [ ]:
# Libraries

import sys
sys.path.append('../data/ma-bench/')
sys.path.append('../data/tau-bench/')

from mabench.environments.airline.tools.book_reservation import book_reservation
from mabench.environments.airline.tools.calculate import calculate
from mabench.environments.airline.tools.cancel_reservation import cancel_reservation
from mabench.environments.airline.tools.get_reservation_details import get_reservation_details
from mabench.environments.airline.tools.get_user_details import get_user_details
from mabench.environments.airline.tools.list_all_airports import list_all_airports
from mabench.environments.airline.tools.search_direct_flight import search_direct_flight
from mabench.environments.airline.tools.search_onestop_flight import search_onestop_flight
from mabench.environments.airline.tools.send_certificate import send_certificate
from mabench.environments.airline.tools.think import think
from mabench.environments.airline.tools.transfer_to_human_agents import transfer_to_human_agents
from mabench.environments.airline.tools.update_reservation_baggages import update_reservation_baggages
from mabench.environments.airline.tools.update_reservation_flights import update_reservation_flights
from mabench.environments.airline.tools.update_reservation_passengers import update_reservation_passengers

domain = "airline"

from tau_bench.envs.airline.wiki import WIKI

## REWOO-ReAct Hybrid Orchestration

REWOO-ReAct reframes *"how tools are used"* rather than *"which tools exist."* We keep a single tool-executor for all airline APIs, but we enforce a **plan → execute** separation around it. In Strands, this becomes a compact, explicit graph where each node returns a typed result (`AgentResult`) and the runtime forwards those results downstream in a deterministic way. This leads to **governance**, **observability**, and **repeatability** while maintaining execution flexibility.

### Create PLANNER: Receives user query and makes the plan

Now let's create the first component of our REWOO-ReAct hybrid system - the **Planner tool**. This is where the *"Reasoning without Observation"* happens. The Planner's job is to create structured execution plans without actually running any tools, providing guidance for the ReAct solver.

#### Planner specific tool

We first define a `generate_flight_plan` tool that takes a user query and converts it into a step-by-step plan using the **#E1, #E2, #E3** format. The tool includes a comprehensive prompt that:

- Lists all available airline tools
- Provides detailed examples of how to structure plans for different scenarios like flight changes, new bookings, and passenger updates
- Supports **REPEAT blocks** for handling batch operations like processing multiple reservations

We create a specialized planning agent that uses this prompt along with airline policy knowledge from the **WIKI** to generate reliable, structured and policy compliant plans that the ReAct Solver agent can execute with flexibility and adaptation capabilities.


In [ ]:
def direct_llm_call(prompt):
    max_tokens = 2048
    temp = 0
    topP = 1
    response = get_claude_response(user_message=prompt,
                                    model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0", 
                                    max_tokens=max_tokens, 
                                   temp=temp)                
    answer = get_claude_response_text(response)
    return answer


In [ ]:
@tool
def generate_flight_plan(user_query: str) -> str:
    """Generate a structured flight plan for the given user query"""

    print(f"\n INSIDE generate_flight_plan tool \n")
    # planner prompt
    planner_prompt = """
# PLANNING ONLY ASSISTANT - DO NOT EXECUTE

Your ONLY job is to write a plan using the exact format below. You MUST NOT try to execute the plan or have any other interactions.

## Available Flight Tools
* calculate[expression]
* get_reservation_details[reservation_id]
* update_reservation_flights[reservation_id, cabin, flights, payment_id]
* search_onestop_flight[origin, destination, date]
* send_certificate[user_id, amount]
* cancel_reservation[reservation_id]
* search_direct_flight[origin, destination, date]
* get_user_details[user_id]
* list_all_airports[]
* book_reservation[user_id, origin, destination, flight_type, cabin, flights, passengers, payment_methods, total_baggages, nonfree_baggages, insurance]
* think[thought]
* transfer_to_human_agents[summary]
* update_reservation_passengers[reservation_id, passengers]
* update_reservation_baggages[reservation_id, total_baggages, nonfree_baggages, payment_id]
* book_reservation[user_id, origin, destination, flight_type, cabin, flights, passengers, payment_methods, total_baggages, nonfree_baggages, insurance]
* cancel_reservation[reservation_id]
* calculate[expression]

## REPEAT Syntax and Usage
When multiple iterations of the same steps are needed, use this format:

1. First, use think tool to analyze and count items to process
2. Then, use another think tool to plan iteration details
3. Finally, use REPEAT block with the count from previous steps

REPEAT(count_from_previous_step) {
    tool1[parameters]
    tool2[parameters]
    ...
}

Available variables in REPEAT blocks:
- CURRENT_ITERATION (0-based index)
- CURRENT_ITEM (from list being processed)
- Other variables extracted from previous steps

Use REPEAT blocks when:
- Processing multiple reservations
- Applying multiple certificates
- Handling multiple passengers
- Any task that requires the same steps multiple times

Note: Evidence numbers inside REPEAT will be expanded sequentially


## Required Format - USE EXACTLY THIS:

Plan 1: [Description]
#E1 = [tool_name][parameters]

Plan 2: [Description]
#E2 = [tool_name][parameters]

## Examples:


Example 1 : "Can you put me on an earlier flight? My reservation ID is 'CD789012'"
Plan 1: Retrieve the current reservation details
#E1 = get_reservation_details[reservation_id="CD789012"]

Plan 2: Search for earlier direct flights based on the origin, destination and date from #E1
#E2 = search_direct_flight[origin=origin_airport_code, destination=destination_airport_code, date=travel_date]

Plan 3: Update the reservation with the earlier flight found in #E2 and use details from #E1 and useer query as necessary
#E3 = update_reservation_flights[reservation_id="CD789012", cabin=cabin_class, flights=selected_flights, payment_id=payment_info]

Example 2 : "My user id is mia_li_3668. I want to fly from New York to Seattle on May 20 (one way). I do not want to fly before 11am EST. I want to fly in economy. I prefer direct flights but one stopover is also fine. If there are multiple options, I prefer the one with the lowest price. I have 3 baggages. I do not want insurance. I want to use my two certificates to pay. If only one certificate can be used, I prefer using the larger one, and pay the rest with my 7447 card"
Plan 1: Get user details to check available certificates
#E1 = get_user_details[user_id="mia_li_3668"]

Plan 2: Get list of airports to find the airport codes for New York and Seattle
#E2 = list_all_airports[]

Plan 3: Search for direct flights using airport codes from #E2 and date from given user question
#E3 = search_direct_flight[origin=origin_airport_code, destination=destination_airport_code, date=travel_date]

Plan 4: If no suitable direct flights, search for one-stop flights using using airport codes from #E2 and date from given user question
#E4 = search_onestop_flight[origin=origin_airport_code, destination=destination_airport_code, date=travel_date]

Plan 5: Return selected flights from #E4  and #E3 


Example 3 : "I have a booking number TR7845. I need to update my daughter's name from Emma Wilson to Emma Thompson as she recently got married. I'm Jennifer Wilson, ID: TW5432P891."

Plan 1: Retrieve the current reservation details
#E1 = get_reservation_details[reservation_id="TR7845"]

Plan 2: Verify user identity and authorization
#E2 = get_user_details[user_id="TW5432P891"]

Plan 3: Think about the passenger name changes needed
#E3 = think["Analyze the user query and reservation details:

Identify the passenger whose name needs to be changed: Emma Wilson
New name for this passenger: Emma Thompson
Keep all other passengers unchanged
Preserve existing passenger details (DOB, etc) from reservation
Create an updated passenger list with the name change"]

Plan 4: Update the reservation with modified passenger information
#E4 = update_reservation_passengers[reservation_id="TR7845", passengers=[
{"first_name": "Jennifer", "last_name": "Wilson", "dob": extracted_dob_jennifer},
{"first_name": "Emma", "last_name": "Thompson", "dob": extracted_dob_emma}
]]

Example 4 : "Hi, my name is Jordan Smith (customer ID: ZX7890Y123). I have a reservation with booking code LM5678 for a flight from Chicago to Miami on June 15. 
I need to add my son, Alex Smith, to the reservation and include an extra bag for him. Can you help me with that?"

Plan 1: Retrieve the current reservation details
#E1 = get_reservation_details[reservation_id="LM5678"]

Plan 2: Verify user identity and authorization
#E2 = get_user_details[user_id="ZX7890Y123"]

Plan 3: Think about required passenger updates
#E3 = think["Analyze current reservation and requested changes:

Get existing passenger list from #E1
New passenger to add: Alex Smith (son)
Need to preserve all existing passenger details
Create updated passenger list that includes both existing and new passengers"]

Plan 4: Update the reservation with complete passenger list
#E4 = update_reservation_passengers[reservation_id="LM5678", passengers=[
extract_existing_passengers_from_E1,
{"first_name": "Alex", "last_name": "Smith", "type": "child"}
]]

Plan 5: Think about baggage update
#E5 = think["Calculate baggage updates:

Get current total_baggages from #E1
Add one extra bag for new passenger
Determine if extra bag is free or paid based on cabin class"]

Plan 6: Update the baggage count
#E6 = update_reservation_baggages[
reservation_id="LM5678",
total_baggages=current_total_plus_one,
nonfree_baggages=current_nonfree_plus_one,
payment_id=payment_from_context
]

Example 5: "My user id is ABC123. I want to downgrade all my business flights to economy for my reservations. Please calculate total savings."

Plan 1: Get user details to retrieve all reservations
#E1 = get_user_details[user_id="ABC123"]

Plan 2: REPEAT(length_of_reservations_from_#E1) {
    get_reservation_details[reservation_id=CURRENT_RESERVATION_ID]    
    calculate["current_savings = business_fare - economy_fare"]
    calculate["total_savings += current_savings"]
    update_reservation_flights[reservation_id=CURRENT_RESERVATION_ID, cabin="economy", flights=CURRENT_FLIGHTS, payment_id=CURRENT_PAYMENT]
}


## IMPORTANT: 
1. ONLY write the plan - nothing else
2. Do NOT add any explanations or clarifications
3. Do NOT attempt to execute any actions
4. Follow the format exactly as shown
5. Use 'think' tool only when needed like name change.


<policy>
{policy}
</policy>
"""
    
    planning_llm = Agent(
        model=bedrock_model_taubench,
        system_prompt=planner_prompt.replace("{policy}", WIKI)
    )
    plan = planning_llm(user_query)
    
    return str(plan)



### Create SOLVER: Receives plan and executes with ReAct flexibility

Now let's implement the **Solver Agent tool** that executes the structured plan using ReAct reasoning with direct tool access. This represents our hybrid approach where we combine REWOO's planning structure with ReAct's execution adaptability.

#### Solver specific tool

The `solve_flight_query` tool takes the structured plan from the Planner and executes it using a ReAct agent that has:

- **Full access to all airline tools** for direct execution including booking, searching, updating reservations, and managing passengers
- **Plan-guided execution** that follows the structured steps (#E1, #E2, #E3) while maintaining ReAct's think→act→observe flexibility
- **Policy-aware execution** that can adapt to real-world constraints and airline business rules from the WIKI
- **Error handling and recovery** capabilities when plans need adjustment or policy violations occur
- **Natural language synthesis** of results for customer-friendly responses

The solve prompt enforces strict discipline around plan execution while allowing ReAct flexibility within each step. It ensures tools are only called as authorized in the plan, validates arguments against policy constraints, and produces concise user-facing responses that summarize what actions were taken and their outcomes.

This hybrid approach maintains the systematic planning benefits of REWOO while gaining the execution flexibility and error recovery of ReAct, creating a more robust and efficient two-agent system that can handle complex airline customer service scenarios with both reliability and adaptability.



In [ ]:

tools=[book_reservation,
        calculate,
        cancel_reservation,
        get_reservation_details,
        get_user_details,
        list_all_airports,
        search_direct_flight,
        search_onestop_flight,
        send_certificate,
        think,
        transfer_to_human_agents,
        update_reservation_baggages,
        update_reservation_flights,
        update_reservation_passengers
      ],


solve_prompt = """# Operating Mode (Plan-Guided ReAct)
- You must execute ONLY the tools and steps authorized in <plan>, in order (#E1..#En). Do not invent new steps or tools.
- Within each step you may think→act→observe (ReAct), but:
  - Use concrete, non-placeholder arguments only.
  - Resolve cross-step references from prior results (#E*), the user task, and the policy.
  - Validate argument types/enums; infer airport states (e.g., “Houston → Texas”) and ensure city↔state consistency.
  - If a step cannot run (policy or missing data), STOP that branch and produce a policy-aligned outcome (e.g., suggest cancel→rebook for Basic Economy).

# Tool & Policy Discipline
- Never call tools not listed in <plan>. Never change the step order.
- Before any mutating call (book/update/cancel/refund/charge):
  - Check fare class, change/refund windows, baggage/insurance rules, payment priorities, and user authorizations.
  - If the user request is **clear and in-policy**, proceed without redundant confirmation.
  - If ambiguity or irreversible risk exists, ask exactly one targeted clarification before acting.
- Do not fabricate IDs, flight numbers, prices, or payment methods.
- Keep side effects idempotent: avoid double updates/charges; summarize deltas precisely (what changed, totals, sources of funds).

# Output Contract
- Return only what the user needs: the answer and/or a concise confirmation with key details (itinerary deltas, amounts, instruments used).
- You must reveal internal thoughts, tool traces, or step logs unless explicitly asked.

# Execution Template 
for step in plan(#E1..#En):
  think (in <think> tags) → call tool with validated args → observe → update working memory
after last step:
  synthesize final user-facing answer that reflects policy, tool outputs, and the task


<policy>
{policy}
</policy>

<plan>
{plan}
</plan>

Response:
"""


@tool
def solve_flight_query(plan: str, user_query: str) -> str:
    """
    Solve user query using structured evidence from worker execution
    """

    try:

        
        formatted_prompt = solve_prompt.format(plan=plan, policy=WIKI)
        agent = Agent(tools=tools, system_prompt=formatted_prompt)
        print(f" *********** starting rewoo guided react response ************* \n" )
        react_response = agent(user_query)   
        
        result = str(react_response)
        return result
        
    except Exception as e:
        print(f"DEBUG: Exception in solve_flight_query: {e}")
        print(f"DEBUG: Exception type: {type(e)}")
        raise

## Custom Agent Classes for Multiagent Graph

Now let's implement the **custom agent classes** that extend the base Strands Agent to work with our multiagent graph. We create these custom classes so that each agent can properly integrate with the graph's streaming architecture while maintaining their specialized roles in the hybrid REWOO-ReAct pattern.

### PlannerAgent Implementation

The `PlannerAgent` class serves as the entry point for our hybrid system, generating structured plans without executing any tools. We implement this with a custom `stream_async` method so that it can seamlessly integrate with the multiagent graph while providing the structured output format required for the ReAct solver.

The agent normalizes the input prompt and calls its `generate_flight_plan` tool to create the structured execution plan. It then wraps the result in the proper `AgentResult` format with message content and metrics that the multiagent graph expects. We structure it this way so that the graph can easily pass the plan to the next agent in the workflow.

### SolverAgent Implementation

The `SolverAgent` class handles the plan execution using ReAct reasoning with full tool access. We implement this with sophisticated input parsing so that it can extract both the original user query and the structured plan from the multiagent graph's combined input.

The agent uses the `extract_original_task_and_plan` helper function to separate the user's original request from the planner's structured output. It then calls the `solve_flight_query` tool with both components, allowing the ReAct agent to execute the plan while maintaining flexibility to adapt based on intermediate results and policy constraints.

### Agent Instantiation

We create instances of both custom agents with their respective tools and configurations. The `planner_agent` uses only the `generate_flight_plan` tool to maintain separation of concerns, while the `solver_agent` uses the `solve_flight_query` tool that has access to all airline operations through its internal ReAct implementation.

This design demonstrates how the Strands framework can be extended to support sophisticated multi-agent reasoning patterns while maintaining clean separation between planning and execution phases.

In [ ]:
# Build rewoo graph
class PlannerAgent(Agent):
    async def stream_async(self, prompt: str):
        # Call the tool and get result
        print(f"DEBUG: PLANNER AGENT CALLED \n")
        prompt=normalize_prompt(prompt)
        plan_result = self.tool.generate_flight_plan(user_query=prompt)
        
        # Create  AgentResult object with required parameters
        message = Message(content=[{"text": str(plan_result)}])
        metrics = EventLoopMetrics()
        
        agent_result = AgentResult(
            stop_reason="end_turn",
            message=message,
            metrics=metrics,
            state=None
        )
        
        # Yield the result event that multiagent graph expects
        yield {"result": agent_result}

# Use Custom planner
planner_agent = PlannerAgent(
    model=bedrock_model_taubench,
    tools=[generate_flight_plan],
    name="planner"
)


class SolverAgent(Agent):
    async def stream_async(self, prompt: str):
        # Extract plan and evidence from the combined input
        # The prompt will contain both original task and worker results
        print(f"DEBUG: SOLVER AGENT CALLED TO FORM FINAL ANSWER FROM EXECUTED PLAN\n")
        prompt=normalize_prompt(prompt)
        parsed = extract_original_task_and_plan(prompt)
        print("ORIGINAL TASK:\n", parsed["original_task"], "\n")
        print("PLAN TEXT:\n", parsed["plan_text"], "\n")
        # Call solve_flight_query tool
        final_answer = self.tool.solve_flight_query(
            user_query=parsed["original_task"],
            plan=parsed["plan_text"]
            
        )
        
        # Create AgentResult object
        message = Message(content=[{"text": str(final_answer)}])
        metrics = EventLoopMetrics() # check how to get the eventloopmetrics
        
        agent_result = AgentResult(
            stop_reason="end_turn",
            message=message,
            metrics=metrics,
            state=None
        )
        
        yield {"result": agent_result}


# Use the custom solver
solver_agent = SolverAgent(
    model=bedrock_model_taubench,
    tools=[solve_flight_query],
    name="solver"
)



## Graph Construction and Orchestration

Now let's create the **REWOO-ReAct multiagent graph** that orchestrates the interaction between our Planner and Solver agents. This function builds the complete hybrid workflow using the Strands GraphBuilder to create a seamless planning and execution pipeline.

### Graph Builder Configuration

The `create_rewoo_react_graph` function uses the Strands `GraphBuilder` to construct our two-agent hybrid system. We add both the `planner_agent` and `solver_agent` as nodes in the graph with their respective identifiers, allowing the graph to route messages and maintain execution flow between the planning and execution phases.

### Sequential Flow Design

We establish the connection between agents using `add_edge("planner", "solver")`, creating a direct sequential path from the planner's structured output to the solver's ReAct execution. We design it this way so that the hybrid approach maintains the systematic planning benefits of REWOO while enabling the execution flexibility of ReAct.

### Entry Point and Execution

The `set_entry_point("planner")` call designates the planner agent as the starting point for all user queries. We configure it this way so that every interaction begins with structured plan generation, followed by the ReAct-based execution process.

When the graph executes, it follows this streamlined flow:
1. User query enters at the planner agent
2. Planner generates structured execution plan (#E1, #E2, #E3 format)
3. Output automatically routes to solver agent with both original query and plan
4. Solver executes plan using ReAct reasoning with full tool access and policy awareness
5. Final customer-friendly response is returned

This compact two-agent design demonstrates how the Strands multiagent graph framework can efficiently combine different agent patterns, creating a hybrid system that leverages the strengths of both REWOO planning reliability and ReAct execution adaptability while reducing latency compared to traditional three-agent REWOO implementations.

In [ ]:
# Finally create the graph with the 2 agent nodes
def create_rewoo_react_graph():
   
    builder = GraphBuilder()    
    # Add the three agents
    builder.add_node(planner_agent, "planner")    
    builder.add_node(solver_agent, "solver")
    
    # Sequential flow: planner  -> solver
    builder.add_edge("planner", "solver")
    
    builder.set_entry_point("planner")
    return builder.build()

## Load Dataset

Now let's load the **TauBench evaluation dataset** that contains real airline customer service scenarios. We do this so that we can test our hybrid system against standardized benchmarks and measure its performance on authentic customer queries like:

- Flight changes
- Cancellations  
- Booking modifications

This loads the **single-turn airline tasks** from TauBench, which provides us with a collection of customer queries along with their expected outcomes for evaluation purposes.

In [ ]:
output_path = os.path.join("..", "data", "tau-bench", "tau_bench", "envs", f"{domain}", "tasks_singleturn.json")
with open(output_path, "r") as file:
    tasks = json.load(file)


## Testing and Evaluation Framework

Now let's implement our **testing framework** that evaluates the REWOO-ReAct hybrid system performance and provides detailed execution analysis. We create comprehensive testing utilities so that we can measure execution time, track agent interactions, and save detailed results for each test case, enabling thorough evaluation of our hybrid approach.

### Response Processing Utilities

The `extract_text_from_response` function handles the parsing of agent responses from different formats. We implement this with robust error handling so that it can process both JSON and Python literal formats that may be returned by the multiagent graph, ensuring consistent text extraction regardless of the response structure.

### Comprehensive Test Function

The `test_rewoo_react_graph` function creates a complete testing environment that loads test cases from the TauBench dataset and executes them through our hybrid system. We extract the user query and metadata from the selected task so that we can test with realistic airline customer service scenarios.

The function creates a fresh instance of the REWOO-ReAct graph for each test execution and measures the total execution time from start to finish. We capture these performance metrics so that we can evaluate the efficiency gains of our two-agent hybrid approach compared to traditional three-agent REWOO implementations.

### Results Analysis and Logging

The test function provides comprehensive execution analysis by displaying graph status, node completion metrics, and individual agent results. We structure the output so that developers can easily trace the flow from planner's structured plan generation through solver's ReAct execution to the final customer response.

The function also saves detailed results to output files with proper formatting and error handling. We implement this persistent logging so that test results can be analyzed later and compared across different test cases or system configurations.

### Test Execution

Finally, we execute the test with question_id 20 from the TauBench dataset, demonstrating how our REWOO-ReAct hybrid system handles a real airline customer service scenario. This shows the complete flow from planning through execution to final customer response, validating both the technical implementation and practical effectiveness of our hybrid approach.




In [ ]:
# Create and execute
import json
import ast
import time

def extract_text_from_response(response_str):
    try:
        # First try to parse as JSON
        response_dict = json.loads(response_str)
    except json.JSONDecodeError:
        try:
            # If JSON fails, try ast.literal_eval
            response_dict = ast.literal_eval(response_str)
        except:
            return "Error parsing response"
    
    # Extract text from content
    try:
        return response_dict['content'][0]['text']
    except (KeyError, IndexError):
        return "Error extracting text"
        

# previous without getting metrics
def test_rewoo_react_graph(question_id):
    task = tasks[question_id]
    user_query = task["question"]
    user_id = task['user_id']
    session_id= uuid.uuid4()
    rewoo_react_graph = create_rewoo_react_graph()
    start=time.time()
    result = rewoo_react_graph(user_query)
    
    exec_time=time.time()-start
    print("=== REWOO Multiagent Graph Results ===")
    print(f"Graph execution time: {exec_time}")
    #print(f"Metrics: {result.metrics}")
    print(f"Status: {result.status}")
    print(f"Total nodes: {result.total_nodes}")
    print(f"Completed nodes: {result.completed_nodes}")
    

question_id = 20 #20, #48
test_rewoo_react_graph(question_id)

##NOTE: To run other queries please rerun all the other cells

# Sample ReWOO-ReAct Hybrid Framework Execution

## User Query
"My user id is james_taylor_7043. I want to change my upcoming one-stop flight from LAS to IAH to a nonstop flight. My reservation ID is 1N99U6. I also want to remove my checked bag and want the agent to refund me for the same."

## ReWOO-ReAct Hybrid Framework Response

### Planning Phase (ReWOO)
The **Planner Agent** creates a comprehensive 5-step plan:

Copy
Plan 1: Retrieve the current reservation details to understand the booking
#E1 = get_reservation_details[reservation_id="1N99U6"]

Plan 2: Get user details to verify identity and check membership status
#E2 = get_user_details[user_id="james_taylor_7043"]

Plan 3: Search for direct flights from LAS to IAH on the same date as the current reservation
#E3 = search_direct_flight[origin="LAS", destination="IAH", date=travel_date_from_E1]

Plan 4: Think about baggage policy limitations
#E4 = think["Review baggage modification rules:

Users can add checked bags but cannot remove them after booking

This is a system limitation that applies to all reservations

Will need to inform user that baggage removal is not possible"]

Plan 5: Transfer to human agents for baggage removal assistance
#E5 = transfer_to_human_agents[summary="User james_taylor_7043 with reservation 1N99U6 wants to change from one-stop to nonstop LAS-IAH flight and remove checked baggage. Flight change can be processed but baggage removal requires human agent assistance as system does not allow removing bags after booking."]


### Execution Phase (ReAct Style)
The **Solver Agent** executes with conversational reasoning:

**Tool Sequence**:
- **Tool #1**: `get_reservation_details` - "Let me start by retrieving your current reservation details"
- **Tool #2**: `get_user_details` - "Now let me get your user details to verify your membership status"
- **Tool #3**: `search_direct_flight` - "Let me search for nonstop flights from LAS to IAH on your travel date"
- **Tool #4**: `think` - Internal policy analysis
- **Tool #5**: `transfer_to_human_agents` - Escalation for baggage removal

### Final Response
> **Current Reservation Details:**
> - Round-trip economy flights: LAS→PHX→IAH on May 19, and IAH→LAS on May 27
> - 2 passengers (James and Ivan Taylor)
> - 1 checked bag included
> 
> **Available Nonstop Options for May 19:**
> 1. HAT175: Departs 5:00 PM, arrives 8:00 PM - $155 economy
> 2. HAT266: Departs 1:00 PM, arrives 4:00 PM - $146 economy
> 
> **Important Policy Limitation:**
> While I can help you change to a nonstop flight, our system does not allow removing checked bags after booking...
> 
> I've transferred you to a human agent who can assist with both your flight change and the baggage removal request.

### Execution Summary
- **Total execution time**: 25.78 seconds
- **Nodes executed**: 2 (Planner → Solver)
- **Tools used**: 5 (including `think` and `transfer_to_human_agents`)
- **Status**: COMPLETED with proper human escalation

## Key Observations

**What you're seeing in this response correlates to the ReWOO-ReAct hybrid notebook framework:**

- **Structured Planning with Conversational Execution**: Combines ReWOO's systematic approach with ReAct's natural language flow
- **Proactive Policy Analysis**: The `think` step demonstrates embedded reasoning within the structured plan
- **Complete Service Coverage**: Addresses both flight change and baggage removal through appropriate channels
- **Proper Escalation Logic**: Successfully transfers baggage removal to human agents as planned
- **Comprehensive Information Gathering**: Includes user verification and complete reservation analysis
- **Balanced Efficiency**: Achieves thorough coverage while maintaining reasonable execution time (25.78s)

This hybrid approach demonstrates the best of both frameworks - ReWOO's systematic completeness with ReAct's user-friendly communication style.



## Congrats!

Congratulations! You've successfully created and tested a REWOO-ReAct hybrid pattern implementation using Strands multiagent orchestration. This system demonstrates:

- **Two-agent hybrid architecture** combining REWOO's structured planning with ReAct's execution flexibility for optimal performance
- **Plan-guided execution workflow** where structured reasoning (#E1, #E2, #E3 format) guides ReAct tool execution while maintaining adaptability
- **Strands GraphBuilder integration** using custom Agent classes with stream_async() methods for seamless multiagent coordination
- **Best-of-both-worlds approach** leveraging REWOO's planning reliability and ReAct's execution flexibility with reduced latency
- **Airline domain integration** with comprehensive MAbench tool execution and TauBench evaluation datasets for real-world testing

The REWOO-ReAct hybrid pattern excels at complex multi-step tasks requiring both systematic planning and execution adaptability, such as airline customer service scenarios where structured planning ensures comprehensive coverage while ReAct flexibility handles policy constraints, error recovery, and dynamic decision-making during tool execution.

